In [2]:
import numpy as np
import pylab as plt
import pandas as pd
import matplotlib.gridspec as gridspec
from tslearn.metrics import dtw_path

In [3]:
# データの準備
data = pd.read_csv("/Users/yuki-f/Documents/SocSEL/Research/ImageRecognition/546606409.csv", header=None, encoding="shift-jis").values
input_data = pd.read_csv("/Users/yuki-f/Documents/SocSEL/Research/ImageRecognition/546602863.csv", header=None, encoding="shift-jis").values

In [ ]:
def dist(x, y):
    return (x - y)**2

def get_min(m0, m1, m2, i, j):
    if m0 < m1:
        if m0 < m2:
            return i - 1, j, m0
        else:
            return i - 1, j - 1, m2
    else:
        if m1 < m2:
            return i, j - 1, m1
        else:
            return i - 1, j - 1, m2

In [ ]:
def spring(x, y, epsilon):
    Tx = len(x)
    Ty = len(y)

    # C:各マスの累積コスト，　B：最小コストの行/列番号， S:コスト最小のパスの開始位置
    C = np.zeros((Tx, Ty))
    B = np.zeros((Tx, Ty, 2), int)
    S = np.zeros((Tx, Ty), int)

    C[0, 0] = dist(x[0], y[0])

    # 一番左の列は，真っ直ぐ上にコストが累積される
    for j in range(1, Ty):
        C[0, j] = C[0, j - 1] + dist(x[0], y[j])
        B[0, j] = [0, j - 1]
        S[0, j] = S[0, j - 1]
        
    for i in range(1, Tx):
        # 一番下の列のコストは，そのマス自身のコストのみにする
        C[i, 0] = dist(x[i], y[0])
        B[i, 0] = [0, 0]
        # スタート地点の行番号を保持
        S[i, 0] = i
        
        # 上方向にコストを計算
        for j in range(1, Ty):
            pi, pj, m = get_min(C[i - 1, j],
                                C[i, j - 1],
                                C[i - 1, j - 1],
                                i, j)
            C[i, j] = dist(x[i], y[j]) + m
            B[i, j] = [pi, pj]
            # スタート地点の行番号を保持
            S[i, j] = S[pi, pj]
            
        # 最小のDTW距離の終点の行番号
        imin = np.argmin(C[:(i+1), -1])
        
        # 最小のDTW距離
        dmin = C[imin, -1]
        
        # dminがepsilonより大きければ抜ける
        if dmin > epsilon:
            continue
                
        for j in range(1, Ty):
            # コストがdminより小さいかつ,区間が被る場合は，
            # dminは最小コストではなくなるため次へ
            if (C[i,j] < dmin) and (S[i, j] < imin):
                break
                
        else:
            path = [[imin, Ty - 1]]
            temp_i = imin
            temp_j = Ty - 1
            
            while (B[temp_i, temp_j][0] != 0 or B[temp_i, temp_j][1] != 0):
                path.append(B[temp_i, temp_j])
                temp_i, temp_j = B[temp_i, temp_j].astype(int)
                
            C[S <= imin] = 100000000
            yield np.array(path), dmin